# Estimate Wasserstein distance discribing envolutionary changes
Methods in [1] were adopted, whose source code can be found in [2]. Besides, data in 'df_n','df_g' were also downloaded from [2].<br/>
This version was updated in Dec., 2020.

### This code aims to extract Wasserstein distance discribing envolutionary changes from data in Australia National Electricity Market.###

#### Recall data preparation for IEEE RTS 24-node model embeding Australia NEM data [3],[4]: ####

In total, the NEM generator dataset contains technical and economic information relating to 203 generating units （conventional generators and renewable energy units, each has a dispatchable unit ID）, while the network dataset consists of 912 nodes, and 1406 AC edges with line voltages in the range of 110 kV to 500 kV.
There are ***5*** trading regions (NSW,QLD,SA,TAS,VIC) and ***16*** NEM zones in the operation devided by AEMO. To facilitate our transmission network planning study, We take this 16-node model by assuming that the power balance constriants are met in each NEM zone.Therefore, the load and wind power signals are aggragated in these 16 nodes based on their geographic positions.

In the original IEEE RTS 24-node model, it has 17 load nodes while no wind power. <br/>
In the modified IEEE RTS 24-node model, technical and economic information relating to generators and topology of existing lines are not changed. However, to be compatible with the NEM dataset: <br/> (1) load demand at #13 node (reference node) is set to be zero (so that we get a 16-load model);  <br/> (2) #11,#12,#17,#24 are selected as nodes equipped with wind power units.  

#### Note that, in estimating  Wasserstein distance discribing envolutionary changes:####
 
 (1) Alothough a 16-node load vector is considered, we estimate the Wasserstein distance using the load data aggregated at the five trading regions, since the detialed load data at each node is not avialiable;<br />
 (2) Although a 4-node wind power vector is considered, we estimate the Wasserstein distance using data at site "NSA", since wind power at the remaining sites are negligible in early years, say, the year of 2011.

### Basic procedures ###
1. Get the standard version of variable injections, $P_v=[P_D;P_W]$  denoted as $z$;
2. Fit distributions of $z$ as Gaussian mixture models (GMMs);
3. Calculate a Wasserstein-type distance between several 5-year-step GMM pairs, including [2011 v.s. 2016], [2012 v.s. 2017],  [2013 v.s. 2018], [2014 v.s. 2019], [2015 v.s. 2020]. Therefore, envolutionary changes over five years can be described.
    

## Import packages

In [1]:
import os
import pandas as pd
import numpy as np
from zipfile import ZipFile
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

from sklearn import mixture
from sklearn.cluster import MiniBatchKMeans

import scipy.io as sio
from pyomo.environ import *
import matplotlib.pyplot as plt

from random import random
from scipy.stats import multivariate_normal
from scipy.stats import matrix_normal

import matplotlib.pyplot as plt
import scipy.stats as sps
import scipy.linalg as spl
from scipy.optimize import linprog

import ot
from sklearn import preprocessing


##  Paths to directories

In [2]:
# Core data directory (common files)
data_dir = os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, os.path.pardir, 'data'))

# MMSDM data directory
MMSDMdata_dir = os.path.join(data_dir,'AUS2009_2020')

# Network directory
network_dir = os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, '1_network'))

# Generators directory
gens_dir = os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, '2_generators'))

# Output directory
output_dir = os.path.abspath(os.path.join(os.path.curdir, 'output'))
output_for_mat_dir=os.path.abspath(os.path.join(os.path.curdir, 'output_for_mat'))

# basic DataFrame for network (index=NODE_ID)
df_n = pd.read_csv(os.path.join(network_dir, 'output', 'network_nodes.csv'), index_col='NODE_ID', dtype={'NEAREST_NODE':np.int32})

# basic DataFrame for generator (index=DUID)
df_g = pd.read_csv(os.path.join(gens_dir, 'output', 'generators.csv'), dtype={'NODE': int}, index_col='DUID')

# population information 


In [3]:
def Standardization(df_population):
    L=np.linalg.cholesky(df_population.cov().values)
    invL=np.linalg.inv(L)
    
    Zsample=np.dot(invL,(df_population-df_population.mean()).values.T)
    df_z=pd.DataFrame(Zsample).T

    return df_z

## Fit GMM 

These GMM paramters would be used for measuring the "distance" between current probability distribution and future probability distribution

In [4]:
def GMM_fit(df_population):
    # Fit the GMM and save the GMM parameters as distionary and mat. file
    clf = mixture.GaussianMixture(n_components=3, covariance_type='full')
    clf.fit(df_population)

    GMM={}
    GMM['Pr']=clf.weights_# Pr for each components
    GMM['means']=clf.means_
    GMM['Covariances']=clf.covariances_
    return GMM

#### The following codes are downloaded from [5]: Wasserstein-type distance betwwen two GMMs

In [5]:
# Wasserstein distance betwwen two Guassians
def GaussianW2(m0,m1,Sigma0,Sigma1):
    # compute the quadratic Wasserstein distance between two Gaussians with means m0 and m1 and covariances Sigma0 and Sigma1
    Sigma00  = spl.sqrtm(Sigma0)
    Sigma010 = spl.sqrtm(Sigma00@Sigma1@Sigma00)
    d        = np.linalg.norm(m0-m1)**2+np.trace(Sigma0+Sigma1-2*Sigma010)
    return d

In [6]:
# Wasserstein-type distance betwwen two GMMs
def GW2(pi0,pi1,mu0,mu1,S0,S1):
    # return the GW2 discrete map and the GW2 distance between two GMMs
    K0 = mu0.shape[0]
    K1 = mu1.shape[0]
    d  = mu0.shape[1]
    S0 = S0.reshape(K0,d,d)
    S1 = S1.reshape(K1,d,d)
    M  = np.zeros((K0,K1))
    # First we compute the distance matrix between all Gaussians pairwise
    for k in range(K0):
        for l in range(K1):
            M[k,l]  = GaussianW2(mu0[k,:],mu1[l,:],S0[k,:,:],S1[l,:,:])
    # Then we compute the OT distance or OT map thanks to the OT library
    wstar     = ot.emd(pi0,pi1,M)         # discrete transport plan
    distGW2   = np.sum(wstar*M)
    return distGW2

## Duration of the dataset

In [7]:
month_list=['0101','0201','0301','0401','0501','0601','0701','0801','0901','1001','1101','1201']

#year_list1_load=['2009','2010','2011','2012','2013','2014']
#year_list1_wind=['2010','2011','2012','2013','2014']
year_list1=['2011','2012','2013','2014','2015']
year_list2=['2016','2017','2018','2019','2020']

Missing_list=['20090101','20090201','20090301','20090401','20090501','20090601','20150601','20160101','20170701','20170801','20170901','20200601','20200701','20201201']

### Function to get dispatch data of wind power
1. Every dispatched units have an ID (i.e., DUID) while we attached the fuel type from 'df_g', so that wind power dispatch data can be extracted;
2. Parse and save unit dispatch data. Note that dispatch in MW is given at 5min intervals, and that the time resolution of demand data is 30min intervals, corresponding to the length of a trading period in the NEM. To align the time resolution of these signals unit dispatch data are aggregated, with mean power output over 60min intervals computed for each DUID.


In [8]:
# Get the four wind power data
def get_zonal_wind_power (df_DISPATCH_UNIT_SCADA):
    # Convert to datetime objects
    df_DISPATCH_UNIT_SCADA['SETTLEMENTDATE'] = pd.to_datetime(df_DISPATCH_UNIT_SCADA['SETTLEMENTDATE'])
    # Pivot dataframe. Dates are the index values, columns are DUIDs, values are DUID dispatch levels
    df_DISPATCH_UNIT_SCADA_piv = df_DISPATCH_UNIT_SCADA.pivot(index='SETTLEMENTDATE', columns='DUID', values='SCADAVALUE')
    # To ensure the 30th minute interval is included during each trading interval the time index is offset
    # by 1min. Once the groupby operation is performed this offset is removed.
    df_DISPATCH_UNIT_SCADA_agg = df_DISPATCH_UNIT_SCADA_piv.groupby(pd.Grouper(freq='60Min', base=1, label='right')).mean()
    df_DISPATCH_UNIT_SCADA_agg = df_DISPATCH_UNIT_SCADA_agg.set_index(df_DISPATCH_UNIT_SCADA_agg.index - pd.Timedelta(minutes=1))
    df_DISPATCH_UNIT_SCADA_agg
    # (1)Nodal renewable energy system (RES) disaptch;(2)NEM Zonal RES dispatch
    # Add fuel category to each DUID in SCADA dispatch dataframe
    df_DISPATCH_UNIT_SCADA_agg = df_DISPATCH_UNIT_SCADA_agg.T.join(df_g[['FUEL_CAT']])
    # Only consider intermittent solar and wind generators
    mask = df_DISPATCH_UNIT_SCADA_agg['FUEL_CAT'].isin(['Wind'])
    # Keep wind and solar (RES) DUIDs, drop fuel category column, and transpose (columns=DUID, index=Timestamp)
    # All intermittent generation profiles
    # (columns=DUID, index=Timestamps)
    df_DUID_RES = df_DISPATCH_UNIT_SCADA_agg[mask].drop('FUEL_CAT', axis=1).T  

    #(1) Nodal RES
    # Add node to which generator is connected, groupby node and sum, 
    # reindex columns using all node IDs, yielding total intermittent injection at each node
    # Injections from intermittent sources (columns=node ID, index=Timestamps)
    df_nodal_RES=df_DUID_RES.T.join(df_g[['NODE']], how='left').groupby('NODE').sum().T.reindex(columns=df_n.index, fill_value=0)

    df_regional_RES=df_nodal_RES.T.join(df_n[['NEM_REGION']], how='left').groupby('NEM_REGION').sum().T.reindex(columns=df_n['NEM_REGION'].unique(), fill_value=0)

    #(2)Zonal RES
    #Fixed injections from intermittent sources for each NEM zone (columns=Zone ID, index=Timestamps
    df_zonal_RES=df_nodal_RES.T.join(df_n[['NEM_ZONE']], how='left').groupby('NEM_ZONE').sum().T.reindex(columns=df_n['NEM_ZONE'].unique(), fill_value=0)
    
    #(3) return the selected RESs for IEEE 24 model
    df_wind=df_zonal_RES[['NSA']]
    return df_wind

## Function to get variable injections in one year
 Load data in each NEM region are given at 30min intervals. Likewise, demand data are aggregated, with mean values over 60min intervals computed for each region. <br />



In [9]:
def get_STDdata_oneYear(year):
        df_Pv=pd.DataFrame()
        df_load_5Region=pd.DataFrame()
        df_wind=pd.DataFrame(columns = ['NSA']) 
        
        for month in month_list:
            a=year+''+month
            if a in Missing_list:
                continue
            print(a)    
            #-------Load data-----------------
            name_csv='PUBLIC_DVD_TRADINGREGIONSUM_201901010000.CSV'
            name_csv=name_csv.replace('0101',month)
            name_csv=name_csv.replace('2019',year)
            # Regional summary for each trading interval
            df_TRADINGREGIONSUM = pd.read_csv(os.path.join(MMSDMdata_dir, name_csv),
                                              skiprows=1, skipfooter=1, engine='python')
            # Convert settlement date to datetime
            df_TRADINGREGIONSUM['SETTLEMENTDATE'] = pd.to_datetime(df_TRADINGREGIONSUM['SETTLEMENTDATE'])
            # Pivot dataframe. Index is timestamp, columns are NEM region IDs, values are total demand
            df_TRADINGREGIONSUM_piv = df_TRADINGREGIONSUM.pivot(index='SETTLEMENTDATE', columns='REGIONID', values='TOTALDEMAND')
            df_TRADINGREGIONSUM_piv = df_TRADINGREGIONSUM_piv.groupby(pd.Grouper(freq='60Min', base=1, label='right')).mean()
            df_TRADINGREGIONSUM_piv = df_TRADINGREGIONSUM_piv.set_index(df_TRADINGREGIONSUM_piv.index - pd.Timedelta(minutes=1))
            
            df_load_5Region=df_load_5Region.append(df_TRADINGREGIONSUM_piv)
            
            #-------Wind data-----------------
            name_csv='PUBLIC_DVD_DISPATCH_UNIT_SCADA_201901010000.CSV'
            name_csv=name_csv.replace('0101',month)
            name_csv=name_csv.replace('2019',year)
        
            df_DISPATCH_UNIT_SCADA = pd.read_csv(os.path.join(MMSDMdata_dir, name_csv),
                                         skiprows=1, skipfooter=1, engine='python')
    
            df_wind0=get_zonal_wind_power (df_DISPATCH_UNIT_SCADA)            
            df_wind=df_wind.append(df_wind0)
        

        #-------Variable injections--------
        df_Pv = pd.concat([df_load_5Region, df_wind], axis=1)
            
        # Standardization
        df_z=Standardization(df_Pv)
                    
        return  df_z

In [10]:
def Analyze_norms(df_z,year):
    df_z_norm1=df_z.apply(lambda x: np.linalg.norm(x,ord=1),axis=1)
    df_z_norm2=df_z.apply(lambda x: np.linalg.norm(x,ord=2),axis=1)
            
    df_norm1_DiffR=pd.DataFrame(columns=['1','2','3','4','5','6','7','8'])
    for i in range(1,9):
            mask=df_z_norm2<=i
            z_norm1_sub=df_z_norm1[mask]
            df_norm1_DiffR.loc['norm1_Year='+str(year),str(i)]=np.percentile(z_norm1_sub,95)                 
            df_norm1_DiffR.loc['Pr_Year='+str(year),str(i)]=len(z_norm1_sub)/len(df_z_norm1)
            
    print(df_norm1_DiffR)


In [ ]:
def get_data_oneYear(year):
        df_load_5Region=pd.DataFrame()
        df_wind=pd.DataFrame(columns = ['NSA']) 
        
        for month in month_list:
            a=year+''+month
            if a in Missing_list:
                continue
            print(a)    
            #-------Load data-----------------
            name_csv='PUBLIC_DVD_TRADINGREGIONSUM_201901010000.CSV'
            name_csv=name_csv.replace('0101',month)
            name_csv=name_csv.replace('2019',year)
            # Regional summary for each trading interval
            df_TRADINGREGIONSUM = pd.read_csv(os.path.join(MMSDMdata_dir, name_csv),
                                              skiprows=1, skipfooter=1, engine='python')
            # Convert settlement date to datetime
            df_TRADINGREGIONSUM['SETTLEMENTDATE'] = pd.to_datetime(df_TRADINGREGIONSUM['SETTLEMENTDATE'])
            # Pivot dataframe. Index is timestamp, columns are NEM region IDs, values are total demand
            df_TRADINGREGIONSUM_piv = df_TRADINGREGIONSUM.pivot(index='SETTLEMENTDATE', columns='REGIONID', values='TOTALDEMAND')
            df_TRADINGREGIONSUM_piv = df_TRADINGREGIONSUM_piv.groupby(pd.Grouper(freq='60Min', base=1, label='right')).mean()
            df_TRADINGREGIONSUM_piv = df_TRADINGREGIONSUM_piv.set_index(df_TRADINGREGIONSUM_piv.index - pd.Timedelta(minutes=1))
            
            df_load_5Region=df_load_5Region.append(df_TRADINGREGIONSUM_piv)
            
            #-------Wind data-----------------
            name_csv='PUBLIC_DVD_DISPATCH_UNIT_SCADA_201901010000.CSV'
            name_csv=name_csv.replace('0101',month)
            name_csv=name_csv.replace('2019',year)
        
            df_DISPATCH_UNIT_SCADA = pd.read_csv(os.path.join(MMSDMdata_dir, name_csv),
                                         skiprows=1, skipfooter=1, engine='python')
    
            df_wind0=get_zonal_wind_power (df_DISPATCH_UNIT_SCADA)            
            df_wind=df_wind.append(df_wind0)
        

            #-------Variable injections--------
            df_Pv = pd.concat([df_load_5Region, df_wind], axis=1)
            
            # Standardization and analyze
            df_z=Standardization(df_Pv)
                    
            # Fit GMM
            GMM_z=GMM_fit(df_z)
            
        return GMM_z

## Main function 

In [15]:
distGMM_set = np.zeros((5,3))
for i in range(0,5):
        y1=year_list1[i]
        y2=year_list2[i]    

        
        df_z1=get_STDdata_oneYear(y1)
        GMM_z1=GMM_fit(df_z1)
        Analyze_norms(df_z1,y1)
        
        df_z2=get_STDdata_oneYear(y2)
        GMM_z2=GMM_fit(df_z2)
        Analyze_norms(df_z2,y2)
        
        
        
        pi0=GMM_z1['Pr']
        pi1=GMM_z2['Pr']
        mu0=GMM_z1['means']
        mu1=GMM_z2['means']
        
        if y1=='2012':
            S0=GMM_z1['Covariances']-1e-6
            S1=GMM_z2['Covariances']-1e-6
        else:
            S0=GMM_z1['Covariances']
            S1=GMM_z2['Covariances']        

        distGW2=GW2(pi0,pi1,mu0,mu1,S0,S1)
        distGMM_set[i,0]=y1
        distGMM_set[i,1]=y2
        distGMM_set[i,2]=sqrt(distGW2)
        
        print(sqrt(distGW2))

20110101
20110201
20110301
20110401
20110501
20110601
20110701
20110801
20110901
20111001
20111101
20111201
                        1         2         3         4         5         6  \
norm1_Year=2011   2.13493   4.23609   5.97689   6.85137   7.25929   7.41814   
Pr_Year=2011     0.010274  0.359589  0.867123  0.964612  0.986644  0.994292   

                        7         8  
norm1_Year=2011   7.51801    7.5747  
Pr_Year=2011     0.997831  0.999429  
20160201
20160301
20160401
20160501
20160601
20160701
20160801
20160901
20161001
20161101
20161201
                         1         2        3         4         5         6  \
norm1_Year=2016    2.15646   4.23131  5.92391   6.89868   7.37691   7.61424   
Pr_Year=2016     0.0110867  0.351365  0.86172  0.965765  0.989035  0.996954   

                        7         8  
norm1_Year=2016    7.6649   7.67938  
Pr_Year=2016     0.999025  0.999756  
1.0598880712990255
20120101
20120201
20120301
20120401
20120501
20120601
20120701
2012080

#### Summary of final simulation results

In [19]:
df_distGMM_set=pd.DataFrame(distGMM_set)
df_distGMM_set.columns=['Year1','Year2','Wasserstein-type distance betwwen two GMMs']
df_distGMM_set

,Year1,Year2,Wasserstein-type distance betwwen two GMMs
0,2011.0,2016.0,1.059888
1,2012.0,2017.0,1.425779
2,2013.0,2018.0,1.311492
3,2014.0,2019.0,0.625847
4,2015.0,2020.0,1.447591


#### Mean value of Wasserstein-type distance betwwen two GMMs, which is used in our simulation

In [20]:
np.mean(distGMM_set[:,2])

1.1741195023888609

# References
[1] -Xenophon, A., Hill, D. Open grid model of Australia’s National Electricity Market allowing backtesting against historic data. Sci Data 5, 180203 (2018). https://doi.org/10.1038/sdata.2018.203

[2] -Xenophon, A. K. Geospatial Modelling of Australia’s National Electricity Market. GitHub https://github.com/akxen/egrimod-nem (2018).

[3] -Australian Energy Markets Operator. Data Archive (2018). at http://www.nemweb.com.au/#mms-data-model 

[4] -Australian Energy Markets Operator. Data Archive (2018). at https://nemweb.com.au/Data_Archive/Wholesale_Electricity/MMSDM/


[5] https://github.com/judelo/gmmot
#  Datasets List from MMSDM
A summary of the tables used from AEMO's MMSDM database [3] is given below:

| Table | Description |
| :----- | :----- |
|DISPATCH_UNIT_SCADA | MW dispatch at 5 minute (dispatch) intervals for DUIDs within the NEM.|
|TRADINGREGIONSUM | Contains load in each NEM region at 30 minute (trading) intervals.|